In [13]:
from openai import OpenAI
import pandas as pd
import json
import time

In [17]:
client = OpenAI(
    api_key="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",
    base_url="https://openrouter.ai/api/v1"
)

df = pd.read_csv("llm_vergleich_resultate_100_1.csv")



PermissionDeniedError: Error code: 403 - {'error': {'message': 'Key limit exceeded. Manage it using https://openrouter.ai/settings/keys', 'code': 403}}

In [15]:
# === Funktion: Produktbeschreibung aus Gruppe (Parquet) erzeugen ===
def create_product_description(group):
    lines = []
    for _, row in group.iterrows():
        line = f"{row['PropertyDefinition']}: {row['PropertyValue']}"
        if pd.notnull(row['UnitName']):
            line += f" {row['UnitName']}"
        lines.append(line)
    return "\n".join(lines)

# === Funktion: Prompt an LLM schicken ===
def run_prompt(model, prompt):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=700
    )
    return response.choices[0].message.content

# === Bewertungs-Prompt mit fixer Modellreihenfolge erstellen ===
def create_evaluation_prompt(product_id, outputs, original_prompt, original_product_text):
    prompt = f"""
Du bekommst die Antworten von vier Modellen auf folgenden Prompt:

{original_prompt}

Die Antworten kommen immer in folgender Reihenfolge:

1. GPT-4  
2. Claude-3  
3. Llama-3  
4. Mistral

Hier sind die Original-Produktinformationen aus der Parquet-Datei als Referenz:

{original_product_text}

Bewerte bitte jede JSON-Antwort hinsichtlich:

- Ist das JSON technisch valide?
- Stimmen die Werte mit den Originaldaten überein?
- Sind alle wichtigen Produktspezifikationen enthalten?
- Sind die Attributnamen sinnvoll und konsistent?
- Wie gut ist die Antwort insgesamt?

Produkt {product_id}:

Modell 1 (GPT-4) Antwort:
{outputs.get('gpt-4', 'Keine Antwort')}

Modell 2 (Claude-3) Antwort:
{outputs.get('claude-3-opus', 'Keine Antwort')}

Modell 3 (Llama-3) Antwort:
{outputs.get('llama-3-70b', 'Keine Antwort')}

Modell 4 (Mistral) Antwort:
{outputs.get('mistral-7b', 'Keine Antwort')}

Bewerte jetzt bitte.
"""
    return prompt

# === Hauptprogramm ===
def main():
    # CSV mit den Modellantworten laden
    df_results = pd.read_csv("llm_vergleich_resultate_100_1.csv")

    # Parquet-Datei mit Originaldaten laden und gruppieren
    df_data = pd.read_parquet("241212_processedproperties_fhnw.parquet")
    grouped_data = df_data.groupby("ProductId")

    # Modelle fest in Reihenfolge definieren
    model_columns = [
        "output_gpt-4",
        "output_claude-3-opus",
        "output_llama-3-70b",
        "output_mistral-7b"
    ]

    evaluation_results = []

    for idx, row in df_results.iterrows():
        product_id = row["ProductId"]
        original_prompt = row["Prompt"]

        # Originaldaten für das Produkt aus Parquet holen
        if product_id in grouped_data.groups:
            group = grouped_data.get_group(product_id)
            original_product_text = create_product_description(group)
        else:
            original_product_text = "Keine Originaldaten gefunden."

        # Outputs in fester Reihenfolge aus dem CSV extrahieren
        outputs = {
            "gpt-4": row.get("output_gpt-4", "Keine Antwort"),
            "claude-3-opus": row.get("output_claude-3-opus", "Keine Antwort"),
            "llama-3-70b": row.get("output_llama-3-70b", "Keine Antwort"),
            "mistral-7b": row.get("output_mistral-7b", "Keine Antwort"),
        }

        # Bewertungs-Prompt erzeugen
        eval_prompt = create_evaluation_prompt(product_id, outputs, original_prompt, original_product_text)

        try:
            print(f"Bewerte Produkt {product_id} ...")
            evaluation = run_prompt("openai/gpt-4", eval_prompt)
            evaluation_results.append({"ProductId": product_id, "Evaluation": evaluation})
            time.sleep(1.5)  # um Rate Limits zu vermeiden
        except Exception as e:
            evaluation_results.append({"ProductId": product_id, "Evaluation": f"Fehler: {str(e)}"})

    # Ergebnisse speichern
    pd.DataFrame(evaluation_results).to_csv("llm_evaluation_results.csv", index=False)
    print("✅ Bewertung gespeichert in 'llm_evaluation_results.csv'.")

In [16]:
if __name__ == "__main__":
    main()

Bewerte Produkt 200440 ...
Bewerte Produkt 203240 ...
Bewerte Produkt 204460 ...
Bewerte Produkt 207900 ...
Bewerte Produkt 213020 ...
Bewerte Produkt 217400 ...
Bewerte Produkt 217880 ...
Bewerte Produkt 217920 ...
Bewerte Produkt 218940 ...
Bewerte Produkt 219760 ...
Bewerte Produkt 220700 ...
Bewerte Produkt 223540 ...
Bewerte Produkt 226180 ...
Bewerte Produkt 230800 ...
Bewerte Produkt 234280 ...
Bewerte Produkt 235640 ...
Bewerte Produkt 236020 ...
Bewerte Produkt 236480 ...
Bewerte Produkt 239420 ...
Bewerte Produkt 242680 ...
Bewerte Produkt 243500 ...
Bewerte Produkt 245840 ...
Bewerte Produkt 246660 ...
Bewerte Produkt 246740 ...
Bewerte Produkt 247340 ...
Bewerte Produkt 248360 ...
Bewerte Produkt 248500 ...
Bewerte Produkt 257900 ...
Bewerte Produkt 258140 ...
Bewerte Produkt 258300 ...
Bewerte Produkt 260700 ...
Bewerte Produkt 262100 ...
Bewerte Produkt 265340 ...
Bewerte Produkt 266020 ...
Bewerte Produkt 268560 ...
Bewerte Produkt 279960 ...
Bewerte Produkt 280680 ...
B